In [10]:
%%time
import pandas as pd
import numpy as np
import re
from collections import Counter
import string

mainData = pd.DataFrame(pd.read_table('Dataset\smsspamcollection\SMSSpamCollection.txt'))
mainData.rename(index=str, columns={"ham": "Status", "Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...": "Message"})
trainingData, cvData, testData  = np.split(mainData,[int(0.6*len(mainData)),int(0.8*len(mainData))])
spamTraining  = trainingData[trainingData['ham'] == 'spam']
hamTraining  =  trainingData[trainingData['ham'] == 'ham']

spamFreq = []
hamFreq = []
spamWords = ''
hamWords = ''

#Making all words a string-----------------
for i in range(0,len(spamTraining)):
    x = spamTraining['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'].iloc[i]
    x.lower()
    x = re.sub(r'[^\w\s]','',x)
    spamWords = spamWords + " " +x

for i in range(0,len(hamTraining)):
    x = hamTraining['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'].iloc[i]
    x.lower()
    x = re.sub(r'[^\w\s]','',x)
    hamWords = hamWords + " " +x
#-------------------------------------------

#Converting to Dictionary--------------------------
spamFreq = dict(Counter(spamWords.split()).most_common())
hamFreq = dict(Counter(hamWords.split()).most_common())
values = spamFreq.values()
spamValues = list(spamFreq.values())
hamValues = list(hamFreq.values())
spamKeys = list(spamFreq.keys())
hamKeys = list(hamFreq.keys())

for i in range(0,len(spamValues)):
    spamValues[i] = (spamValues[i])/(len(spamValues))
    
for i in range(0,len(hamValues)):
    hamValues[i] = (hamValues[i])/(len(hamValues))
                                    
spamFreq = dict(zip(spamKeys,spamValues))  #Contains Probabilities of all the words in Spam
hamFreq = dict(zip(hamKeys,hamValues))  #Contains Probabilities of all the words in Ham

#Checking Accuracy of Cross Validation Data ------------
cpSpam = 1
cpHam  = 1
answers = []
correct = 0
pSpam = np.log(len(spamTraining)/len(trainingData))
pHam  = np.log(len(hamTraining)/len(trainingData))
for i in range(0,len(cvData)):
    s = (cvData['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'].iloc[i])
    s = s.lower()
    s = re.sub(r'[^\w\s]','',s)
    s = s.split()
    sSpam = list(set(s) & set(spamFreq.keys()))
    cpSpam = 0
    sHam = list(set(s) & set(hamFreq.keys()))
    cpHam = 0
    for j in range(0,len(sHam)):
        cpHam = cpHam + np.log(hamFreq[sHam[j]])
    for j in range(0,len(sSpam)):
        cpSpam = cpSpam + np.log(spamFreq[sSpam[j]])
    fpSpam = (cpSpam+pSpam)-(pHam)
    fpHam = (cpHam+pHam)-(pHam)

    if(fpHam>fpSpam):
        answers.append('spam')
    else:
        answers.append('ham')
     
for i in range(0,len(answers)):
    if(answers[i] == cvData['ham'].iloc[i]):
        correct = correct + 1
print('Cross Validation:---------')
print("CP Spam Log Probability: ",cpSpam)
print("CP Ham Log Probability:",cpHam)
print("Accuracy CV Data: ",(correct/len(answers))*100)


correct = 0
answers = []
pSpam = np.log(len(spamTraining)/len(trainingData))
pHam  = np.log(len(hamTraining)/len(trainingData))
for i in range(0,len(cvData)):
    s = (testData['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'].iloc[i])
    s = s.lower()
    s = re.sub(r'[^\w\s]','',s)
    s = s.split()
    sSpam = list(set(s) & set(spamFreq.keys()))
    cpSpam = 0
    sHam = list(set(s) & set(hamFreq.keys()))
    cpHam = 0
    for j in range(0,len(sHam)):
        cpHam = cpHam + np.log(hamFreq[sHam[j]])
    for j in range(0,len(sSpam)):
        cpSpam = cpSpam + np.log(spamFreq[sSpam[j]])
    fpSpam = (cpSpam+pSpam)-(pHam)
    fpHam = (cpHam+pHam)-(pHam)

    if(fpHam>fpSpam):
        answers.append('spam')
    else:
        answers.append('ham')

for i in range(0,len(answers)):
    if(answers[i] == testData['ham'].iloc[i]):
        correct = correct + 1
print('Testing:-------------')
print("CP Spam Log Probability: ",cpSpam)
print("CP Ham Log Probability:",cpHam)
print("Accuracy Test Data: ",(correct/len(answers))*100)


Cross Validation:---------
CP Spam Log Probability:  -108.126232804
CP Ham Log Probability: -241.607608533
Accuracy CV Data:  75.58348294434471
Testing:-------------
CP Spam Log Probability:  -81.5747501748
CP Ham Log Probability: -112.863429998
Accuracy Test Data:  78.63554757630162
Wall time: 5.9 s
